In [2]:
import os
import cv2
import numpy as np
from typing import Dict, Tuple

In [20]:
def generateImageSetDict(imageSetNum: str, NIR: bool):
    # Initialize outputs
    imgMask = {}
    imgArray = {}
    
    # Initialize Data Path
    imageType = 'NIR' if NIR else 'RED'
    sampleDir = '/home/mark/DataBank/probav_data/train/'
    imageDir = os.path.join(sampleDir, imageType, imageSetNum)
    if not os.path.exists(imageDir):
        print("Path does not exist")
        return ({}, {})
    
    # Iterate through all items and populate the Dicts
    imageList = sorted(os.listdir(imageDir))
    imgArray['HR'] = cv2.imread(os.path.join(imageDir, 'HR.png'))
    imgMask['HR'] = cv2.imread(os.path.join(imageDir, 'SM.png'))
    for img in imageList:
        if img == 'HR.png':
            continue
        elif img == 'SM.png':
            continue
        elif img[0:2] == 'LR':
            imgArray[img.split('.')[0]] = cv2.imread(os.path.join(imageDir, img))
        else:
            imgMask[img.split('.')[0]] = cv2.imread(os.path.join(imageDir, img))
    
    return (imgArray, imgMask)        

In [31]:
qMapImgDir = os.path.join(sampleDir, imageList[22])
qMap = cv2.imread(qMapImgDir, 0)
qMap.shape

(128, 128)

In [27]:
qMapImgDir

'C:\\Users\\Mark\\Documents\\Projects\\databank\\train\\NIR\\imgset0594\\QM000.png'

In [21]:
generateImageSetDict('imgset0888', True)

({'HR': array([[[31, 31, 31],
          [32, 32, 32],
          [33, 33, 33],
          ...,
          [32, 32, 32],
          [35, 35, 35],
          [35, 35, 35]],
  
         [[31, 31, 31],
          [31, 31, 31],
          [32, 32, 32],
          ...,
          [29, 29, 29],
          [30, 30, 30],
          [32, 32, 32]],
  
         [[33, 33, 33],
          [32, 32, 32],
          [33, 33, 33],
          ...,
          [30, 30, 30],
          [30, 30, 30],
          [31, 31, 31]],
  
         ...,
  
         [[32, 32, 32],
          [33, 33, 33],
          [32, 32, 32],
          ...,
          [43, 43, 43],
          [39, 39, 39],
          [34, 34, 34]],
  
         [[32, 32, 32],
          [31, 31, 31],
          [30, 30, 30],
          ...,
          [37, 37, 37],
          [38, 38, 38],
          [33, 33, 33]],
  
         [[31, 31, 31],
          [29, 29, 29],
          [28, 28, 28],
          ...,
          [31, 31, 31],
          [37, 37, 37],
          [35, 35, 35]]], d